In [9]:
import numpy as np
import pandas as pd
from root_pandas import read_root
import glob
import os

from helpfunction import CheckBorderTPC,CheckBorderFixed

In [10]:
pd.options.display.max_columns = 999
gr      = 1.618
nan     = -2147483648
min_root_size = 20000 # Skip root files smaller than x bytes

mass_p= 0.93827 #GeV
mass_e= 0.00511 #GeV

# LAr EM showers
R_moliere =  9.5 # cm
X_o       = 13.9 # cm
E_c       = 0.035# GeV

# Fiducial volume borders in x,y,z:
fid_arr= [[10,10],[20,20],[10,50]]

# Minimum reconstructable energies:
min_e = 0.02+mass_e # 20MeV
min_p = 0.04+mass_p # 40MeV

In [18]:
checkana=True
gpvmsdir = '/run/user/1000/gvfs/sftp:host=uboonegpvm02.fnal.gov,user=wvdp'

if not checkana:
    # Option 1: no checkana, verify validity yourself and use input directory
    inputlist = []
    #inputlist += glob.glob('/home/wouter/Templates/Roberto_nue/*/*.root')
    inputlist += glob.glob(gpvmsdir+'/uboone/app/users/wvdp/Larsoft/srcs/1e1pFilter/pandoraElectronFilter/job/*.root')
    filelist  = []
    for fname in inputlist:
        if (os.stat(fname).st_size)<min_root_size*1:
                print("File",fname,"was corrupt. Size:",os.stat(fname).st_size/1000, "kb, skipping to next file.")
        else:
            filelist.append(fname)

if checkana:
    # Option 2: After checkana, just pass the filesana list.
    filelist = [gpvmsdir+line.rstrip() for line in open(gpvmsdir+'/uboone/data/users/wvdp/book/v06_26_01_12/lee_signal_1s0t/filesana.list')]

print(len(filelist),"valid ROOT files collected.")

9 valid ROOT files collected.


In [19]:
# Signal Definition 1e0p 
def true_thresholds_1e0p(row):
    if CheckBorderTPC(*row[["true_vx_sce","true_vy_sce","true_vz_sce"]],array=fid_arr):
        passed_e=False
        for pdg,E in zip(*row[["nu_daughters_pdg","nu_daughters_E"]]):
            if pdg==11 and E>min_e:
                return True
    return False


In [20]:
fidvol=0
passed=0
flashpassed=0
signal=0
total=0
category=np.zeros(8)

for i_f,fname in enumerate(filelist):
    #print(fname)
    dftemp=read_root(fname,"wouterNueCC/pandoratree",columns=["true_vx_sce","true_vy_sce","true_vz_sce",'vx','vy','vz',
                                                             'passed','category','flash_time',"nu_daughters_pdg","nu_daughters_E",'flash_passed'])
    total+=len(dftemp.index)
    
    dftemp = dftemp[dftemp.apply(true_thresholds_1e0p,axis=1)]
    signal+=len(dftemp.index)
    
    flashpassed+=len(dftemp[dftemp["flash_passed"].apply(lambda x: np.any(x!=-1))].index)
    
    dftemp=dftemp[dftemp["passed"]==1]
    passed+=len(dftemp.index)
    
    
    dftemp=dftemp[dftemp.apply(lambda row: CheckBorderTPC(*row[["vx","vy","vz"]],array=fid_arr),axis=1)]
    fidvol+=len(dftemp.index)
    
    print(np.bincount(dftemp['category'].values))
    category+=np.bincount(dftemp['category'].values)
    
print(total)
print(signal)
print(flashpassed)
print(passed)
print(fidvol)

print(flashpassed/signal,"+/-",np.sqrt(flashpassed)/passed)
print(passed/signal,"+/-",np.sqrt(passed)/passed)
print(fidvol/signal,"+/-",np.sqrt(fidvol)/passed)
print(category[2]/signal,"+/-",np.sqrt(category[2])/passed)

print(category/sum(category))

[ 0  5 68  0  0  0  0 19]
[  0   3 114   0   0   0   0  32]
[ 0  1 16  0  0  0  0  6]
[  0  11 112   0   0   0   0  26]
[ 0  3 68  0  0  0  0 16]
[ 0  2 21  0  0  0  0  6]
[  0   8 143   0   0   0   0  29]
[  0   4 100   0   0   0   0  20]
[  0   8 129   0   0   0   0  23]
4900
1794
1538
1142
993
0.8573021181716833 +/- 0.0343409309127
0.6365663322185061 +/- 0.0295914978086
0.5535117056856187 +/- 0.0275936099065
0.429765886288 +/- 0.0243142616058
[ 0.          0.04531722  0.77643505  0.          0.          0.          0.
  0.17824773]


In [8]:
print(total)
print(signal)
print(flashpassed)
print(passed)
print(fidvol)

print(flashpassed/signal,"+/-",np.sqrt(flashpassed)/passed)
print(passed/signal,"+/-",np.sqrt(passed)/passed)
print(fidvol/signal,"+/-",np.sqrt(fidvol)/passed)
print(category[2]/signal,"+/-",np.sqrt(category[2])/passed)

print(category/sum(category))

175000
43978
40511
0.9211651280185548 +/- 0.00559325957591
35985
0.818250034107963 +/- 0.00527156112327
[ 0.          0.03201334  0.71777129  0.          0.          0.          0.
  0.25021537]
